In [3]:
import pandas as pd
import matplotlib.pyplot as plt
#from google.colab import files
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.tree import export_graphviz
import graphviz

In [4]:
#arquivo=files.upload()
cidade=pd.read_csv("brasilrev2022.csv")

In [5]:
cidade.columns.tolist()

['CITY',
 'STATE',
 'CAPITAL',
 'IBGE_RES_POP',
 'IBGE_RES_POP_BRAS',
 'IBGE_RES_POP_ESTR',
 'IBGE_DU',
 'IBGE_DU_URBAN',
 'IBGE_DU_RURAL',
 'IBGE_POP',
 'IBGE_1',
 'IBGE_1-4',
 'IBGE_5-9',
 'IBGE_10-14',
 'IBGE_15-59',
 'IBGE_60+',
 'IBGE_PLANTED_AREA',
 'IBGE_CROP_PRODUCTION_$',
 'IDHM Ranking 2010',
 'IDHM',
 'IDHM_Renda',
 'IDHM_Longevidade',
 'IDHM_Educacao',
 'LONG',
 'LAT',
 'ALT',
 'PAY_TV',
 'FIXED_PHONES',
 'AREA',
 'REGIAO_TUR',
 'CATEGORIA_TUR',
 'ESTIMATED_POP',
 'RURAL_URBAN',
 'GVA_AGROPEC',
 'GVA_INDUSTRY',
 'GVA_SERVICES',
 'GVA_PUBLIC',
 'GVA_TOTAL',
 'TAXES',
 'GDP',
 'POP_GDP',
 'GDP_CAPITA',
 'GVA_MAIN',
 'MUN_EXPENDIT',
 'COMP_TOT',
 'COMP_A',
 'COMP_B',
 'COMP_C',
 'COMP_D',
 'COMP_E',
 'COMP_F',
 'COMP_G',
 'COMP_H',
 'COMP_I',
 'COMP_J',
 'COMP_K',
 'COMP_L',
 'COMP_M',
 'COMP_N',
 'COMP_O',
 'COMP_P',
 'COMP_Q',
 'COMP_R',
 'COMP_S',
 'COMP_T',
 'COMP_U',
 'HOTELS',
 'BEDS',
 'Pr_Agencies',
 'Pu_Agencies',
 'Pr_Bank',
 'Pu_Bank',
 'Pr_Assets',
 'Pu_Assets',
 '

In [6]:
#modelo de regressão, não será usado (manter até data da entrega final)

#valores padronizados para IDH baixo: de 0 a 0,314, médio: de 0,315 a 0,818, alto: de 0,819 a 1


cidade.drop(index=cidade[cidade['IDHM'] ==0 ].index,inplace=True)

minimo=cidade['IDHM'].min()
maximo=cidade['IDHM'].max()
maximo,minimo

#o menor idh eh 0.418 e o maior eh 0.862



def padronizar_idh (valorAtual, minimo, maximo):
  return (valorAtual - minimo) / (maximo - minimo)

funcao_de_padronizar=cidade['IDHM_Padronizado']=cidade['IDHM'].apply(lambda x: padronizar_idh(x, minimo, maximo))

muito_baixo=0.499 #aqui é o valor maximo da categoria, acima disso ela muda
baixo=0.599
medio=6.999
alto=0.799
muito_alto=1



cidade[['CITY','STATE','IDHM_Padronizado']].head(30)
minimo_padronizado=cidade['IDHM_Padronizado'].min()
maximo_padronizado=cidade['IDHM_Padronizado'].max()
print("O menor IDH padronizado é {} e o maior é {}".format(minimo_padronizado,maximo_padronizado))


X=cidade[['GDP_CAPITA','IDHM_Educacao','COMP_Q']].values
y=cidade[['IDHM']].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
modelo = DecisionTreeRegressor(max_depth=7)
modelo.fit(X_train, y_train)
y_pred = modelo.predict(X_test)
r2 = r2_score(y_test, y_pred)
print('Coeficiente de determinação (R²):', r2)

cidade[['CITY','IDHM_Padronizado']].head(10)

O menor IDH padronizado é 0.0 e o maior é 1.0
Coeficiente de determinação (R²): 0.9422222484069019


,CITY,IDHM_Padronizado
0,Abadia De Goiás,0.653153
1,Abadia Dos Dourados,0.610360
2,Abadiânia,0.610360
3,Abaetetuba,0.472973
4,Abaeté,0.630631
5,Abaiara,0.472973
6,Abaré,0.353604
7,Abatiá,0.605856
8,Abaíra,0.416667
9,Abdon Batista,0.621622


In [7]:
# aqui define classes para ser usadas de parametro
classes = {
    'baixo': 0,
    'médio': 1,
    'alto': 2
}

# 0.315 a 0.818, alto: de 0.819 a 1

# aqui ele é utilizado um controle de fluxo para definir qual valor é retornado na classe, para ver se o idh vai ser médio, alto etc
cidade['IDHM_Classes'] = cidade['IDHM_Padronizado'].apply(lambda x: 'baixo' if x < 0.315 else 'alto' if x > 0.819 else 'médio')
cidade['IDHM_Classes'] = cidade['IDHM_Classes'].map(classes)

# Aqui eh pra separar os dados em treino e teste
X = cidade[['GDP_CAPITA','IDHM_Educacao','COMP_Q','COMP_O','COMP_E','IDHM_Longevidade']].values
y = cidade[['IDHM_Classes']].values.ravel()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42) #tamanho do teste ta em 30%

# aqui controi a arvore de devisao e a profundidade dela
modelo = DecisionTreeClassifier(max_depth=3)
modelo.fit(X_train, y_train)
y_pred = modelo.predict(X_test)

# verifica a precisao do modelo
acc = accuracy_score(y_test, y_pred)
print('Acurácia:', acc)

conf_matrix = confusion_matrix(y_test, y_pred)
print(conf_matrix)

# aqui visualiza graficamente a arvore
dot_data = export_graphviz(modelo, out_file=None,
                           feature_names=['GDP_CAPITA','IDHM_Educacao','COMP_Q','COMP_O','COMP_E','IDHM_Longevidade'],
                           class_names=['Baixo', 'Médio', 'Alto'],
                           filled=True, rounded=True, special_characters=True)


Acurácia: 0.9539198084979055
[[  99   36    0]
 [  18 1471   10]
 [   0   13   24]]


In [8]:
import sklearn.metrics as metrics
print("Relatório sobre a qualidade:\n")
print(metrics.classification_report(y_test, y_pred, target_names=['Baixo', 'Médio', 'Alto']))

Relatório sobre a qualidade:

              precision    recall  f1-score   support

       Baixo       0.85      0.73      0.79       135
       Médio       0.97      0.98      0.97      1499
        Alto       0.71      0.65      0.68        37

    accuracy                           0.95      1671
   macro avg       0.84      0.79      0.81      1671
weighted avg       0.95      0.95      0.95      1671



In [ ]:
from sklearn import tree
import graphviz
dot_data = tree.export_graphviz(modelo, out_file=None)
graph = graphviz.Source(dot_data)
graph.render("cidade")
dot_data = tree.export_graphviz(modelo, out_file=None,
# feature_names=cidade.feature_names,
# class_names=cidade.target_names,
filled=True, rounded=True,
special_characters=True)
graph = graphviz.Source(dot_data, format="png")
graph

In [10]:
# solicita o nome da cidade
nome_cidade = str(input("Digite o nome da cidade: "))

# Filtrar o DataFrame para obter as informações da cidade especificada pelo usuário
cidade_selecionada = cidade.loc[cidade['CITY'].str.upper() == nome_cidade.upper()] # ta essa cara e resposavel por encontrar a cidade nas colunas tem que da toUpper no nome sempre ?

coluna_cidade=cidade[['CITY']] # sim entendi blz  a isso aqui da para pegar as colunas da cidade

cidade['CITY']=cidade['CITY'].str.upper()
# verifica a classe de IDH atribuída à cidade selecionada # @Lucas ta online ???? nao entendi ? se os modelos entao funcionando para receber parametros
if cidade_selecionada.empty:
    print("Cidade não encontrada.")
else:
    # Verifica a classe do IDH de acordo com a cidade filtrada # kkk verdade eu lembrei essa aqui ta adaptado para o modelo de arvore sim para o comite vai ter que mudar um pouqinho blz, so adaptar isso aqui la e passar a cidade e as colunas para os tres modelos votarem
    if cidade_selecionada['IDHM_Classes'].iloc[0] == 0:
        print("A cidade '{}' que você selecionou tem IDH baixo.".format(nome_cidade.upper())) # sim
    elif cidade_selecionada['IDHM_Classes'].iloc[0] == 1:
        print("A cidade '{}' que você selecionou tem IDH médio.".format(nome_cidade.upper()))
    elif cidade_selecionada['IDHM_Classes'].iloc[0] == 2:
        print("A cidade '{}' que você selecionou tem IDH alto.".format(nome_cidade.upper()))

A cidade 'IGARATÁ' que você selecionou tem IDH médio.
